---
### Step 0: Load The Data

In [ ]:
# Import libraries
import math
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
#from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Lambda, Conv2D, Activation, Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

In [ ]:
#img_data_set = []
#label_data_set = []
lines = []
with open('labeled_images.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for line in reader:
            lines.append(line)
print(len(lines))

In [ ]:
#labeled_pictures = pd.read_csv('labeled_images.csv')
shuffle(lines)
train_data, val_data = train_test_split(lines, test_size = 0.2)
print(len(train_data))
print(len(val_data))

In [ ]:
'''
images = []
labels = []
for batch_line in val_data:
    img = cv2.cvtColor(cv2.imread('./rec_images/'+str(batch_line[0])+'.jpeg'), cv2.COLOR_BGR2RGB)
    images.append(img)
    if int(batch_line[1]) == 0: label = 1
    else: label = 0
    #label = 0
    labels.append(label)
'''

In [ ]:
'''
index = 527
plt.imshow(images[index])
print(labels[index])    
print(images[index].shape)
'''

---
### Step 1: Define Data Generator

In [ ]:
def data_generator (data_lines, batch_size = 128) :
    num_lines = len(data_lines)
    while True:
        shuffle(data_lines)
        for offset in range(0, num_lines, batch_size):
            batch_lines = data_lines[offset:offset + batch_size]
            images = []
            labels = []
            for batch_line in batch_lines:
                img = cv2.cvtColor(cv2.imread('./rec_images/'+str(batch_line[0])+'.jpeg'), cv2.COLOR_BGR2RGB)
                images.append(img)
                if int(batch_line[1]) == 0: label = 1
                else: label = 0
                labels.append(to_categorical(label))          
            X = np.array(images)
            y = np.array(labels)
            yield shuffle(X, y)

---
### Step 2: Define Model

In [ ]:
model = Sequential()
#normalize data
model.add(Lambda(lambda x: (x / 255.0) - 0.5,  input_shape=(600,800,3)))
#layer 1- Convolution, no of filters- 24, filter size= 5x5, stride= 2x2
model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="elu"))
#layer 2- Convolution, no of filters- 36, filter size= 5x5, stride= 2x2
model.add(Conv2D(36, (5,5), strides=(2, 2), activation="elu"))
#layer 3- Convolution, no of filters- 48, filter size= 5x5, stride= 2x2
model.add(Conv2D(48, (5,5), strides=(2, 2), activation="elu"))
#layer 4- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64, (3,3), activation="elu"))
#layer 5- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64, (3,3), activation="elu"))
#flatten image from 2D to side by side
model.add(Flatten())
#layer 6- fully connected layer 1
model.add(Dense(100, activation="elu"))
#dropout layer to avoid overfitting
model.add(Dropout(0.25))
#layer 7- fully connected layer 1
model.add(Dense(50, activation="elu"))
#layer 8- fully connected layer 1
model.add(Dense(10, activation="elu"))
#layer 9- fully connected layer 1
model.add(Dense(1))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.fit_generator(generator = data_generator(train_data),
                    validation_data = data_generator(val_data),
                    epochs = 2,
                    steps_per_epoch  = math.ceil(len(train_data) / 128),
                    validation_steps = math.ceil(len(val_data)   / 128)    )
model.save('model.h5')
print('Model saved')